# TwoLayerNetクラスのテキストの写経と幾つかの試行(P114~)

## TwoLayerNetクラスの写経

In [1]:
#最初のお約束の設定
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
#import common.gradient as g #例えば、本書の共通ライブラリを読み込む場合の設定
import numpy as np
import matplotlib.pylab as plt
######################################################

In [2]:
#追加の読み込み設定
from common.functions import *
from common.gradient import numerical_gradient

In [3]:
class TwoLayerNetMy:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        ## 【W1層】
        ## インプットは、(１行、インプット列)の行列。それに対して、W1層は、
        ## （インプット行、hidden_size列）の行列で、インプットの列数と同じ行数で受ける必要がある。
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        ## 【W2層】
        ## W1層の出力が（インプット行、hidden_size列）の行列になる。これがそのまま、W2層へのインプットとなる。
        ## それに対して、アウトプットを(１行、output_size列)の行列に変換する必要があるから、自ずと、W2は、
        ## (hidden_size行、output_size列)の行列になる。
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        temp = sigmoid(np.dot(x, self.params['W1']) + self.params['b1'])
        return  softmax(np.dot(temp, self.params['W2']) + self.params['b2'])
        
    #x: 入力データ、 t:教師データ
    def loss(self, x, t):
        return cross_entropy_error(self.predict(x), t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        #最も確率の高い要素のインデックスを取得。ただし、複数行になるため、行毎(axis=1)にＭＡＸなインデックスを得る
        #このため、zの要素数はy.shape[0]になる。
        y1 = np.argmax(y, axis=1)
        t1 = np.argmax(t, axis=1)
        
        accuracy = np.sum( y == t ) #予測のインデックス==教師データの正解インデックスとなる要素をブロードキャストで計算する。
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

## 写経終了。実際に試していく

In [4]:
#パラメータのサイズの確認
net = TwoLayerNetMy(input_size = 784, hidden_size = 100, output_size = 10)
print(net.params["W1"].shape)
print(net.params["b1"].shape)
print(net.params["W2"].shape)
print(net.params["b2"].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [5]:
#傾きのパラメータサイズの確認
x = np.random.rand(100, 784) # ダミーの入力データ(784ピクセルの画像が100枚分)
t = np.random.rand(100, 10)  # ダミーの正解ラベルが100枚分

grads = net.numerical_gradient(x,t)

print("show grads")
print(grads["W1"].shape)
print(grads["b1"].shape)
print(grads["W2"].shape)
print(grads["b2"].shape)
print("===========")

show grads
(784, 100)
(100,)
(100, 10)
(10,)


ここまで出来るようになったが、一言言えるのは、numerical_gradientの計算がめちゃくちゃ遅いということ。大体、２〜３分かかる（ＣＰＵを１５個くらい１００％利用率で動作させて）

## ミニバッチ学習の実装(P118の写経)

In [6]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet #せっかくなので、テキストのTwoLayerNetを使ってしまおう

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

#ハイパーパラメータの設定
iters_num     = 10
train_size    = x_train.shape[0] #訓練データの個数(画像の枚数)
batch_size    = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

for i in range(iters_num):
    #バッチの状況をiters_numの10%毎に報告する
    print("batch %d done"%(i) )
    if ((i / iters_num) * 100) % 10 == 0:
        print("batch %d done"%(i) )
    
    #ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size) #0~train_size未満の整数の中からランダムにbatch_size個の整数を生成
    x_batch = x_train[batch_mask] #batch_maskで指定した訓練データを取得する
    t_batch = t_train[batch_mask] #正解ラベルも同様に実施
    
    #勾配の計算
    print("calc grad")
    grad = network.numerical_gradient(x_batch, t_batch)
    #誤差逆伝搬法のほうがずっと高速になるらしい！
    #grad = network.gradient(x_batch, t_batch)
    
    #パラメータの更新
    print("update params")
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    #学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

batch 0 done
batch 0 done
calc grad
update params
batch 1 done
batch 1 done
calc grad
update params
batch 2 done
batch 2 done
calc grad
update params
batch 3 done
batch 3 done
calc grad
update params
batch 4 done
batch 4 done
calc grad
update params
batch 5 done
batch 5 done
calc grad
update params
batch 6 done
batch 6 done
calc grad
update params
batch 7 done
batch 7 done
calc grad
update params
batch 8 done
batch 8 done
calc grad
update params
batch 9 done
batch 9 done
calc grad
update params


テキストでは10000イテレーションとなっていたが、１イテレーションあたり2分くらいかかりそうなので、10イテレーションに落とした。

In [7]:
print(train_loss_list)

[2.2889820114450554, 2.2744496324076366, 2.2855275950057408, 2.2975220019498606, 2.2942392060129473, 2.2953792300734426, 2.284913721526043, 2.2715120581118704, 2.2984081760085284, 2.280698811283469]


たった、10イテレーションしか回していないが、ロス率は減っていそうだ。
numerical_gradientは非常に時間がかかるので、誤差逆伝搬法を実装した後で、イテレーションを増やしたい。